# Solving Sudoku Puzzles
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can solve sudoku puzzles. You should be familiar with sudoku puzzles from the unit material. You are given a 9x9 grid with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

For this assignment you will need to submit:
1. The implementation for an agent which can solve sudoku puzzles – this notebook
 * You can use any algorithm you like, from the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned based on whether it can solve sudokus of varying difficulty
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section

### Choice of Algorithm
The choice of algorithm to solve sudoku puzzles is up to you. We expect you will use search techniques from the unit, but you could make something up yourself, or do some independent research to find something else. You will need to evaluate and balance the trade-off between how well suited you think the algorithm is and how difficult it is to write, but there is some advice below.

I suggest you implement *constraint satisfaction* as it is described in the unit material. You can use the code you have previously been given as a guide. A good implementation of a backtracking depth-first search with constraint propagation should be sufficient to get a good grade in the automated tests (roughly 60-70%).

You could also write a successful agent that uses the other search techniques you have seen in the unit so far: basic search, heuristic search, or local search. You may find these easier to implement, though they may perform less well. 

To get a high grade on this assignment will require a particularly efficient implementation of constraint satisfaction, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [1]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [2]:
symbols = '123456789'
cellnames = 'ABCEDFGHI'
debug=False
err_debug=False

def create_cells():
    cell_ids = []
    for c in cellnames:
        for s in symbols:
            cellname= c+s
            cell_ids.append(cellname)
    #print(cell_ids)
    return cell_ids

def create_row_units():
    rowunits = []
    for c in cellnames:
        row_unit = []
        for s in symbols:
            ru = c + s
            row_unit.append(ru)
        rowunits.append(row_unit)

    #print(rowunits)
    return rowunits

def create_col_units():
    columnunits = []
    for s in symbols:
        col_unit = []
        for c in cellnames:
            cu = c + s
            col_unit.append(cu)
        columnunits.append(col_unit)

    #print(columnunits)
    return columnunits

def create_box_units():
    boxunits = []
    for ci in ['123', '456', '789']:
        boxunit = []
        for ri  in ['ABC', 'DEF', 'GHI']:
            box = [row_val + col_val for row_val in ri for col_val in ci]
            boxunit.append(box)
        boxunits.extend(boxunit)
    
    #print(boxunits)
    return boxunits

def create_units():
    unitdict = {}
    allunits = create_col_units() + create_row_units() + create_box_units()
    global cell_ids
    for c in cell_ids:
        cell_items= []
        for unit in allunits:
            if c in unit:
                cell_items.append(unit)

        unitdict[c] =  cell_items 

    #print(unitdict)
    return unitdict


def create_peers():
    peerdict = {}
    global cell_ids
    global units
    for c in cell_ids:
        unit = units[c]
        combine = set(sum(unit,[]))
        peers = combine - set([c])
        peerdict[c] = peers
        
    #print(peerdict)
    return peerdict


def parse_input(sudoku):
    """
    Update possible values of pre-assigned cells and satisfy constraints of peer cells.

    Input
        value_dict : Dictionary of possible values for every cell in the form of a string.
            Empty cells have '123456789'. Filled cells have only one character.

    Output
        Dictionary of cells with consistent partial assignment. False or None if there is no solution.
    """
    possible_val_dict = {}
    global cell_ids
    global symbols
    for c in cell_ids:
        possible_val_dict[c] = symbols

    input_list=[]
    for row in range(9):
        for column in range(9):
            if str(sudoku[row][column]) in symbols:
                input_list.append(str(sudoku[row][column]))
            elif str(sudoku[row][column]) == '0':
                input_list.append('0')
            else:
                if err_debug: print('Error: non numeric input')
                                  
    inputdict=dict(zip(cell_ids,input_list))
    #print(inputdict)
    for cell, setval in inputdict.items():
        if setval in symbols:
            if not set_value(possible_val_dict, cell, setval):
                return False
    #print(possible_val_dict)
    return possible_val_dict
    
def set_value(possible_val_dict, cell, value):
    """
    Assign possible value to a cell by satisfying constraints

    Input
        value_dict : Dictionary of possible values for every cell in the form of a string.
            Empty cells have '123456789'. Filled cells have only one character.

    Output
        Dictionary of cells with consistent partial assignment. False or None if there is no solution.
    """
    
    if debug: print('set_value: '+ value)
    vals_to_remove = possible_val_dict[cell].replace(value, '')
    status = []
    for val in vals_to_remove:
        s = discard(possible_val_dict, cell, val)
        status.append(s)
   
    if all(status):
        return possible_val_dict
    else:
        if err_debug: print('Error: setting value')
        return False


def discard(possible_val_dict, cell, value):
    """
    Remove possible value of a cell from its unit and peers to satisfy constraints

    Input
        value_dict : Dictionary of possible values for every cell in the form of a string.
            Empty cells have '123456789'. Filled cells have only one character.

    Output
        Dictionary of cells with consistent partial assignment. False or None if there is no solution.
    """
    
    if debug: print('discard: '+ value + ' from ' + cell)
    if value not in possible_val_dict[cell]:
        if err_debug: print('value already removed')
        return possible_val_dict
    possible_vals_old = possible_val_dict[cell]
    possible_val_dict[cell] = possible_vals_old.replace(value,'')
    if len(possible_val_dict[cell]) == 0:
        if err_debug: print('Error: removed all possible values for ' + cell)
        return False
    elif len(possible_val_dict[cell]) == 1:
        if debug: print('setting single possible val in discard by removing from peers ' +  possible_val_dict[cell] + ' at ' + cell)
        discard_val = possible_val_dict[cell]
        status = []
        for peer in peers[cell]:
            if debug: print('removing peer for ' + cell + ' value ' + discard_val + ' from ' + peer)
            s = discard(possible_val_dict, peer, discard_val)
            status.append(s)
            
        if not all(status):
            if err_debug: print('Error: removing peers for: '+ cell)
            return False
    global units
    for unit in units[cell]:
        places = []
        for cell in unit:
            if value in possible_val_dict[cell]:
                places.append(cell)
        if len(places) == 0:
            if err_debug: print('Error: no single value place for: '+ cell)
            return False
        elif len(places) == 1:
            if debug: print('setting single value ' + value + ' at ' + places[0])
            if not set_value(possible_val_dict, places[0], value):
                if err_debug: print('Error: setting single value place for: '+ cell)
                return False
            
    return possible_val_dict

            

def get_cell_with_min_values(value_dict):
    """
    Minimum ordering of cell with least number of possible values to use for backtracking.

    Input
        value_dict : Dictionary of possible values for every cell in the form of a string.
            Empty cells have '123456789'. Filled cells have only one character.

    Output
        Cell id of the cell with least number of possible values.
    """
    
    global cell_ids
    min_num_values = 10
    cell_min_num_value = ''
    for cell in cell_ids:
        if len(value_dict[cell]) > 1:
            if len(value_dict[cell]) < min_num_values:
                min_num_values = len(value_dict[cell])
                cell_min_num_value = cell
    return cell_min_num_value
        

def depth_first_search(value_dict):
    """
    Depth first search with backtracking. Try setting possible values for the cell which have minimum number of 
    possible values. Bactrack and retry with different possible value if there is no soution with a possible value. 

    Input
        value_dict : Dictionary of possible values for every cell in the form of a string.
            Empty cells have '123456789'. Filled cells have only one character.

    Output
        Dictionary of cells with consistent and complete assignment. False or None if there is no solution.
    """

    global cell_ids
    possible_vals = []
    if not value_dict:
        if err_debug: print('Error: dict false')
        return False
    
    if all(len(value_dict[cell]) == 1 for cell in cell_ids):
        if err_debug: print(value_dict)
        if err_debug: print('#######solved######')
        return value_dict

    cell = get_cell_with_min_values(value_dict)
    if err_debug: print('*******backtracking setting************')
    if err_debug: print(value_dict[cell])
    backtracking_results=[]
    for possible_val in value_dict[cell]:
        if debug: print('setting value '+ possible_val + ' at cell ' + cell)
        if debug: print('depth_first_search')
        if not value_dict:
            return False
        value_dict_copy = set_value(value_dict.copy(), cell, possible_val)
        result = depth_first_search(value_dict_copy)
        backtracking_results.append(result)

    for result in backtracking_results:
        if result is not False and result is not None:
            return result
        

def get_final_state(value_dict):
    """
    Convert dictionary of cell values (which should be a single character now after solving) to a 9x9 numpy array.

    Input
        value_dict : Dictionary of value for each cell in the form of a char.

    Output
        9x9 numpy array.
    """
    vals = np.zeros([81])
    i=0
    for _, val in value_dict.items():
        if len(val) != 1 or val=='0':
            return -1*np.ones([9,9]).astype(int)
        else:
            vals[i] = val
            i+=1
            
    solved_sudoku = vals.reshape(9,9)
    
    return solved_sudoku

cell_ids = create_cells()
units = create_units()
peers = create_peers()

        
def sudoku_solver(sudoku):
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input
        sudoku : 9x9 numpy array
            Empty cells are designated by 0.

    Output
        9x9 numpy array of integers
            It contains the solution, if there is one. If there is no solution, all array entries should be -1.
    """
       
    val_dict = parse_input(sudoku)
    if not val_dict:
        # Constraint propogation failed
        return -1*np.ones([9,9]).astype(int)
    
    if debug:
        print(val_dict)
        
    val_dict = depth_first_search(val_dict)
    if not val_dict:
           return -1*np.ones([9,9]).astype(int)

    solved_sudoku = get_final_state(val_dict)
        
    
    return solved_sudoku

All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 30 seconds each on the test machine to count as successful, but you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [3]:
SKIP_TESTS = True

if not SKIP_TESTS:
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [4]:
import sys
import pathlib

fail = False;

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the cell above.")
    
p1 = pathlib.Path('./readme.txt')
p2 = pathlib.Path('./readme.md')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("You must include a separate file called readme.txt or readme.md in your submission.")
    
p3 = pathlib.Path('./sudoku.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named sudoku.ipynb")
    
if "sudoku_solver" not in dir():
    fail = True;
    print("You must include a function called sudoku_solver which accepts a numpy array.")
else: 
    sudoku = np.load("data/very_easy_puzzle.npy")[0]
    solution = np.load("data/very_easy_solution.npy")[0]

    if not np.array_equal(sudoku_solver(sudoku), solution):
        print("Warning:")
        print("Your sudoku_solver function does not correctly solve the first sudoku.")
        print()
        print("Your assignment is unlikely to get any marks from the autograder. While we will")
        print("try to check it manually to assign some partial credit, we encourage you to ask")
        print("for help on the forum or directly to a tutor.")
        print()
        print("Please use the readme file to explain your code anyway.")
    
if fail:
    print()
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

All checks passed. When you are ready to submit, upload the notebook and readme file to the
assignment page, without changing any filenames.

If you need to submit multiple files, you can archive them in a .zip file. (No other format.)


In [5]:
# This is a TEST CELL. Do not delete or change.